# Obtaining a list of equities tickers and the corresponding timeseries

In [23]:
# ! pip install --quiet yfinance
# yfinance is already installed in our env virtual environment

In [1]:
import os
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
start_date = '2024-06-01'
end_date  = '2024-09-01'
dates = '{}_{}'.format(start_date, end_date)

data_dir = 'data/'
dates_dir = data_dir + '/' + dates
stock_dir = dates_dir + '/stocks'
aggregated_dir = dates_dir + '/aggregated'

# order matters: outer directories must be created before nested directories
for dir in [data_dir, dates_dir, stock_dir, aggregated_dir]:
  if not os.path.exists(dir):
    os.mkdir(dir)

## Get list of companies in S&P500

We will obtain and store the list of the constituents from Wikipedia.

## Download price timeseries

Now that we have a list of ticker symbols, we can download the corresponding timeseries from Yahoo Finance.

In [3]:
data    = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S%26P_500_component_stocks')
table   = data[0]
symbols = list(table.Symbol.values)
symbols.append('SPY')
symbols.sort()

print("{} symbols in total".format(len(symbols)))

504 symbols in total


In [4]:
pd.options.mode.chained_assignment = None  # default='warn'

success_downloads = 0
failed_downloads = []

for symbol in symbols:
  try:
    df = yf.download(symbol, start=start_date, end=end_date)

    if df.empty:
      failed_downloads.append(symbol)
      print('Failed to download {} data'.format(symbol))
      continue

    df = df[['Open', 'Adj Close', 'Volume']]

    df.to_csv(os.path.join(stock_dir, "{}.csv".format(symbol)))
    success_downloads += 1
  except KeyError:
    print('Error for symbol {}'.format(symbol))
    pass

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

$BF.B: possibly delisted; No price data found  (1d 2024-06-01 -> 2024-09-01)
Failed to download BF.B data



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Failed to download BRK.B data


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [5]:
print('\nSuccessfully stored {}/{} files'.format(success_downloads, len(symbols)))


Successfully stored 502/504 files


In [6]:
# Printing a sample dataframe
idx = np.random.randint(len(symbols))
print("History for {}".format(symbols[idx]))
df = pd.read_csv(os.path.join(stock_dir, symbols[idx]+".csv")).set_index('Date')
df.head()

History for PPL


,Open,Adj Close,Volume
Date,,,
2024-06-03,29.299999,28.763685,4894900
2024-06-04,29.200001,28.822666,4566000
2024-06-05,29.240000,28.449112,4560400
2024-06-06,28.809999,28.213184,4306800
2024-06-07,28.389999,27.928102,4182700


## Pre-process Financial Timeseries

In [7]:
# initialize an empty DateTime Index
index = pd.date_range(start=start_date, end=end_date, freq='D')

# initialize empty dataframes
df_price = pd.DataFrame(index=index, columns=symbols)         # adjusted closing prices
df_volume = pd.DataFrame(index=index, columns=symbols)          # stock volumes
df_returns = pd.DataFrame(index=index, columns=symbols)         # daily (percent) returns

In [8]:
# Aggregate all symbols into a price, volume, daily returns dataframes
for symbol in symbols:
    if symbol in failed_downloads:
      continue

    symbol_df = pd.read_csv(os.path.join(stock_dir, symbol+".csv")).set_index('Date')
    symbol_df.index = pd.to_datetime(symbol_df.index)

    adj_close_p = symbol_df['Adj Close']
    open_p = symbol_df['Open']

    df_price[symbol] = adj_close_p
    df_volume[symbol] = symbol_df['Volume']
    df_returns[symbol] = ((adj_close_p - open_p) / open_p) * 100

# calculate percent change
df_price_pct = df_price.pct_change()[1:]

C:\Users\alain\AppData\Local\Temp\ipykernel_2688\2343861972.py:17: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_price_pct = df_price.pct_change()[1:]
C:\Users\alain\AppData\Local\Temp\ipykernel_2688\2343861972.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_price_pct = df_price.pct_change()[1:]


In [9]:
df_price.head()

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03,131.156586,11.53,193.805664,158.731857,146.250000,102.327126,101.699997,280.529724,439.019989,230.383911,...,254.189957,29.515116,94.598991,54.253155,113.535934,138.242599,138.638260,112.869133,307.630005,171.960968
2024-06-04,130.607620,11.50,194.125305,160.664093,147.080002,102.884171,100.860001,286.802216,448.369995,229.726501,...,254.329483,29.276690,92.615501,54.989220,111.770157,136.736618,140.837265,113.188423,302.769989,171.472137
2024-06-05,133.312592,11.62,195.643539,163.934067,145.779999,102.685226,99.680000,289.749268,455.799988,234.756699,...,252.764725,29.256823,93.054062,54.135380,112.216568,139.299789,139.832291,112.390198,307.500000,175.402786


In [10]:
df_volume.head()

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03,3114800.0,52899000.0,50080500.0,4467800.0,3843400.0,4915100.0,1747000.0,3596600.0,3967700.0,3087800.0,...,370300.0,2974200.0,1197200.0,3195100.0,19589300.0,2466300.0,1618100.0,1662800.0,305600.0,2142700.0
2024-06-04,2893700.0,39161500.0,47471400.0,4215500.0,3048500.0,3880400.0,1217600.0,3498600.0,3183600.0,2616000.0,...,259000.0,2485500.0,1522900.0,3460000.0,19412200.0,1078400.0,2125200.0,1186700.0,219500.0,1304400.0
2024-06-05,2707500.0,34340900.0,54156800.0,5100500.0,3121500.0,3302300.0,1480100.0,3002200.0,3295000.0,4016700.0,...,296100.0,2472400.0,1335000.0,2474900.0,14863500.0,973500.0,1944000.0,918100.0,248000.0,1870800.0


In [11]:
df_returns.head()

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03,0.889681,-0.603454,0.469502,-0.792589,0.349934,0.153790,-0.780491,-2.032576,-1.912507,-2.001823,...,0.224731,-1.616281,-0.943465,-1.928498,-2.376671,-2.094474,1.092504,-1.424338,-2.059854,1.129719
2024-06-04,-0.481854,-0.346620,-0.264434,0.295955,0.864075,0.081875,-1.049740,1.789539,1.253328,-0.920167,...,-0.011998,-1.092266,-1.483354,0.016770,-1.088357,-1.051729,1.045530,0.157881,-0.398060,0.317174
2024-06-05,2.123945,0.955688,0.124639,0.634790,-1.486685,-0.237806,-0.884955,0.516637,1.471534,1.188232,...,-0.564623,-0.790699,0.058131,-3.104742,-0.596538,1.308938,-1.248387,-0.916691,0.885831,1.447538


## Obtain Percentage Change

We need to convert prices to percent change in price as opposed to the actual \$ price. This is because stocks with very similar prices can behave very differently and vice-versa.
For e.g., if a stock moves from \$100 to \$110, we want the price column to say 10% (indicating the change).

However, for volume, we will retain magnitude.

In [12]:
df_price_pct.head()

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
2024-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-04,-0.004186,-0.002602,0.001649,0.012173,0.005675,0.005444,-0.008260,0.022359,0.021297,-0.002854,...,0.000549,-0.008078,-0.020967,0.013567,-0.015553,-0.010894,0.015861,0.002829,-0.015798,-0.002843
2024-06-05,0.020711,0.010435,0.007821,0.020353,-0.008839,-0.001934,-0.011699,0.010276,0.016571,0.021896,...,-0.006152,-0.000679,0.004735,-0.015527,0.003994,0.018745,-0.007136,-0.007052,0.015622,0.022923
2024-06-06,-0.005541,-0.016351,-0.007097,0.018919,0.008369,0.010075,0.004013,0.002474,0.005112,0.007341,...,0.003588,0.010526,-0.009426,-0.006164,0.007514,-0.020047,0.002135,0.006481,-0.000033,0.005460


### Removing NaNs

In [13]:
# Let's drop the dates where all the stocks are NaNs, ie., weekends/holidays where no trading occured
for df in [df_price, df_volume, df_returns, df_price_pct]:
  df.dropna(how='all', inplace=True)
  df.dropna(inplace=True, axis=1)
  print(True in pd.isna(df))

assert((df_price.index == df_volume.index).all())
assert((df_volume.index == df_returns.index).all())
assert((df_returns.index == df_price.index).all())

False
False
False
False


### Storing the cleaned dataframes

In [14]:
df_price.to_csv(os.path.join(aggregated_dir, "prices.csv"), index_label='date')
df_volume.to_csv(os.path.join(aggregated_dir, "volume.csv"), index_label='date')
df_returns.to_csv(os.path.join(aggregated_dir, "percent_return.csv"), index_label='date')
df_price_pct.to_csv(os.path.join(aggregated_dir, "prices_pct.csv"), index_label='date')